In [34]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import git
import xlrd
import matplotlib.pyplot as plt
import random
from scipy.stats import wilcoxon
from sqlalchemy import create_engine, or_, Column, Integer, String, Float, DateTime, ForeignKey, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from do_analysis import Experiment, ExperimentSet

In [3]:
engine = create_engine('sqlite:///results.db')
session_class = sessionmaker(bind=engine)
session = session_class()

# Load data into the dataframe

In [54]:
last = ""
headers = []
datasets = []
series = []
serie = None
repo = git.Repo(search_parent_directories=True)

for experiment in session.query(Experiment).order_by(Experiment.file_name):
#for experiment in session.query(Experiment).filter(or_(Experiment.set_id==i for i in [17])).order_by(Experiment.file_name):
#for experiment in session.query(Experiment).filter(Experiment.number_of_clusters == Experiment.number_of_classes).order_by(Experiment.file_name):
    if last != experiment.file_name:
        last = experiment.file_name
        dataset_name = experiment.file_name.rsplit('/')[-1].split(".")[0]
        datasets.append(dataset_name)
        if serie is not None:
            series.append(serie)
        serie = []
    message = repo.commit(experiment.set.commit).message.strip()
    message = experiment.set.description
    measure =  experiment.method.split('.')[-1]
    if "LearningBased" in measure:
        strategy = measure.split(" ")[4]
        weight = measure.split(" ")[6]
        measure = f"LearningBased strategy {strategy}, weight {weight}"
    header = f"{measure} - {message} - {experiment.set.commit}"
    if header not in headers:
        headers.append(header)
    if experiment.number_of_classes is None or experiment.number_of_clusters < experiment.number_of_classes /2:
        serie.append(np.nan)
    else:
        serie.append(experiment.f_score)


series.append(serie)

df = pd.DataFrame(series, index = datasets, columns=headers)
df

,EskinModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,GambaryanModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,GoodallModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,OFModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,IOFModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,LinModified_Kappa - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,LinModified_KappaMax - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,EskinModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a,GambaryanModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a,GoodallModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a,...,"LearningBased strategy C, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy D, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy D, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy D, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy E, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy E, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy E, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy N, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy N, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy N, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a"
allbp_complete_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.601846,0.829734,0.909016,0.832534,0.815091,0.663440,0.829734,NaN,NaN,NaN
arrhythmia_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.441823,0.427121,0.501966,0.431420,0.425638,0.424684,0.437865,NaN,NaN,NaN
australian_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.727323,0.636628,0.674492,0.727323,0.636628,0.674492,0.727323,0.652566,0.605572,0.727323
cervical_cancer_biopsy_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908,0.926908
cervical_cancer_cytology_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389,0.935389
cervical_cancer_hinselmann_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911,0.945911
cervical_cancer_schiller_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946,0.906946
crx_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.674492,0.674492,0.674492,0.674584,0.674492,0.674492,0.674584,0.671404,0.753200,0.671404
diagnosis1_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.564785,0.567037,0.564785,0.564785,0.567037,0.564785,0.564785,0.567037,0.567037,0.567037
diagnosis2_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.744626,0.643799,0.744626,0.744626,0.643799,0.744626,0.744626,0.643799,0.643799,0.643799


# Rank each value against the other options, per dataset

In [55]:
dft = df.apply(lambda x: x.rank(ascending=False), axis = 1)
dft

,EskinModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,GambaryanModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,GoodallModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,OFModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,IOFModified - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,LinModified_Kappa - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,LinModified_KappaMax - Base measures times kappa - fd316b0863142ae83e94c99489ef5adfde7638ed,EskinModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a,GambaryanModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a,GoodallModified - Base measure times (1-kappa) discarding - 425c2e53b4e31b14a4a32ac25bb76f12d90ca62a,...,"LearningBased strategy C, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy D, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy D, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy D, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy E, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy E, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy E, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy N, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy N, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a","LearningBased strategy N, weight A - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a"
allbp_complete_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50.5,16.5,4.0,13.5,19.5,37.0,16.5,NaN,NaN,NaN
arrhythmia_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,37.5,2.0,30.5,43.5,45.0,16.5,NaN,NaN,NaN
australian_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.5,63.5,29.5,7.5,63.5,29.5,7.5,59.5,66.0,7.5
cervical_cancer_biopsy_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0
cervical_cancer_cytology_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0
cervical_cancer_hinselmann_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0
cervical_cancer_schiller_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0
crx_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50.5,50.5,50.5,23.5,50.5,50.5,23.5,72.5,1.5,72.5
diagnosis1_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59.0,21.0,59.0,59.0,21.0,59.0,59.0,21.0,21.0,21.0
diagnosis2_cleaned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.0,65.0,25.0,25.0,65.0,25.0,25.0,65.0,65.0,65.0


# Get the mean rank per measure and order them from best to worst

In [59]:
averages = dft.mean()
averages.sort_values()

LinModified_KappaMax - Original measures - 390053a96d1201a90e07adbf02db1a1acd1cd006                                                9.896552
LearningBased strategy N, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a                               13.907407
LearningBased strategy N, weight N - Make symmetric - ff56e80408af3411a1ac8f2ad348da15b84318c0                                    16.648148
LearningBased strategy N, weight N - Make symmetric, eliminating by weight strategy - 035fb5fc61793a3b54765aa717a151d95ff3131f    16.648148
LearningBased strategy N, weight N - Discard by Kappa in LearningBased - d85bbad6c034587c701938bcb59542c18de595e7                 17.814815
LearningBased strategy N, weight N - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a                               17.814815
LearningBased strategy N, weight A - Discard by Kappa in LearningBased - d85bbad6c034587c701938bcb59542c18de595e7                 18.629630
LearningBased strate

# Compare the best and second best using the Wilcoxon test

In [60]:
best = averages.sort_values().index[0]
second_best = averages.sort_values().index[1]
_, p = wilcoxon(df[best], df[second_best])
p
#for i, a in zip(range(3), averages.sort_values()):
#    print(type(a))

0.00011890609974290849

# Check the differences between the best and second best

In [61]:
print(f"{best} | {second_best}")
for i, (x, y) in enumerate(zip(df[best], df[second_best])):
    if x != y: 
        print(f"{df.index[i]}: {x} - {y}")

LinModified_KappaMax - Original measures - 390053a96d1201a90e07adbf02db1a1acd1cd006 | LearningBased strategy N, weight K - LearningBased Basic - 6b178d1e59f8a2130d8a53383215e359cf85a09a
allbp_complete_cleaned: 0.95669949054718 - nan
arrhythmia_cleaned: 0.488440066576004 - nan
australian_cleaned: 0.671404480934143 - 0.60557234287262
cervical_cancer_biopsy_cleaned: 0.936098456382751 - 0.926907598972321
cervical_cancer_cytology_cleaned: 0.948796987533569 - 0.935388684272766
cervical_cancer_hinselmann_cleaned: 0.959228694438934 - 0.945910573005676
cervical_cancer_schiller_cleaned: 0.914340972900391 - 0.906946301460266
crx_cleaned: 0.671404480934143 - 0.753199994564056
diagnosis1_cleaned: 0.56478488445282 - 0.567036509513855
diagnosis2_cleaned: 0.725274741649628 - 0.643799483776093
drug_consumption_alcohol_cleaned: 0.424801558256149 - 0.338430732488632
drug_consumption_amphet_cleaned: 0.480038464069366 - 0.442556321620941
drug_consumption_benzos_cleaned: 0.490026444196701 - 0.42873114347457

In [ ]:
def only_upper(s):
    return "".join(c for c in s if c.isupper())


originals = []
for av in averages.index:
    if "Base measures times kappa" in av:
        name = av.split(" ")[0]
        if "OccurenceFrequency" in av:
            name = only_upper(name)
        originals.append((name, averages[av]))
originals

In [ ]:
better_than = {}
for av in averages.index:
    measure = f"{av.split(' ')[0]}"
    for o_name, o_value in originals:
        if f"{o_name}" == measure:
            if o_value > averages[av]:
                if o_name not in better_than:
                    better_than[o_name] = []
                better_than[o_name].append((av, averages[av]))
better_than

In [ ]:
all_colors = list(plt.cm.colors.cnames.keys())
random.seed(1000)
c = random.choices(all_colors, k=125)
fig = plt.figure(figsize=(16,10), dpi= 80)
ax = fig.add_subplot(111)
#fig, ax = plt.subplots()
#box = df.boxplot(ax=ax)

bp = ax.boxplot(df.transpose(), autorange=True, widths=0.65, patch_artist=True)
ax.margins(y=0.05)

for label in (plt.gca().get_yticklabels()):
    label.set_fontsize(18) # Size here overrides font_prop
for i, box in enumerate(bp['boxes']):
    # change outline color
    #box.set( color='#7570b3', linewidth=2)
    # change fill color
    box.set_facecolor(c[i])
    pass
for median in bp['medians']:
    median.set(color='black')
    
for i, val in enumerate(averages.values):
    plt.text(i+1, val, "{0:.4f}".format(val),horizontalalignment='center', verticalalignment='bottom', fontdict={'fontweight':500, 'size':18})
    
plt.ylabel('F1-Score', fontsize=20)
plt.gca().set_xticklabels(df.index, rotation=60, horizontalalignment= 'right', fontdict={'fontweight':500, 'size':18})
plt.savefig("box.png", transparent=True, bbox_inches="tight")
plt.show()

In [ ]:
df.reindex(dft.mean().sort_values().index, axis=1)